In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Paths
model_path = "/workspace/nudenet/Models/alexnet.h5"  # Replace with your model name
test_data_folder = "/workspace/nudenet/NudeNet_Classifier_train_data_x320/preprocessed/nude_sexy_safe_v1_x320/testing"

# Load the model
model = tf.keras.models.load_model(model_path)

# Print model summary
print(f"Model Summary for {os.path.basename(model_path)}:")
model.summary()

# Get the model's input shape
input_shape = model.input_shape[1:3]  # (height, width)
print(f"Model expects input shape: {input_shape}")

# Create test data generator
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
test_data = test_datagen.flow_from_directory(
    test_data_folder,
    target_size=input_shape,
    batch_size=32,
    class_mode="binary",
    shuffle=False
)

# Predict on test data
y_true = test_data.classes
y_pred_probs = model.predict(test_data)
y_pred = (y_pred_probs > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_probs)

# Print metrics
print("\nEvaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


2024-11-17 11:27:19.761191: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-17 11:27:19.792019: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-17 11:27:21.240099: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-17 11:27:21.271692: I external/local_

Model Summary for alexnet.h5:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization (Batch  (None, 55, 55, 96)        384       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_1 (Bat  (None, 27, 27, 256)       1024      
 chNormalization)                                                
                        

2024-11-17 11:27:26.749461: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 90500


248/248 [==============================] - 9s 21ms/step

Evaluation Metrics:
Accuracy: 0.8604
Precision: 0.8521
Recall: 0.8793
F1-Score: 0.8655
ROC-AUC: 0.9351


In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Paths
models_folder = "/workspace/nudenet/Models"
test_data_folder = "/workspace/nudenet/NudeNet_Classifier_train_data_x320/preprocessed/nude_sexy_safe_v1_x320/testing"

# Initialize results storage
results = []

# Loop through models
for model_name in os.listdir(models_folder):
    if model_name.endswith(".h5"):
        model_path = os.path.join(models_folder, model_name)
        
        # Load the model
        model = tf.keras.models.load_model(model_path)
        
        # Get model's expected input shape
        input_shape = model.input_shape[1:3]  # (height, width)
        print(f"Processing model: {model_name} | Expected input shape: {input_shape}")
        
        # Create test data generator with the model's input shape
        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
        test_data = test_datagen.flow_from_directory(
            test_data_folder,
            target_size=input_shape,
            batch_size=32,
            class_mode="binary",
            shuffle=False
        )
        
        # Predict
        y_true = test_data.classes
        y_pred_probs = model.predict(test_data)
        y_pred = (y_pred_probs > 0.5).astype(int)  # Convert probabilities to binary labels

        # Check shapes and types
        print(f"y_true shape: {y_true.shape}, y_pred shape: {y_pred.shape}")
        print(f"y_true sample: {y_true[:5]}, y_pred sample: {y_pred[:5]}")

        # Ensure that both y_true and y_pred are in binary format
        if y_true.ndim > 1:
            y_true = np.argmax(y_true, axis=1)  # Flatten multi-label targets if any

        # Compute metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        roc_auc = roc_auc_score(y_true, y_pred_probs)  # roc_auc_score can handle probabilities directly

        # Append results
        results.append({
            "Model": model_name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1,
            "ROC-AUC": roc_auc
        })


# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save to CSV
results_csv_path = "model_evaluation_results.csv"
results_df.to_csv(results_csv_path, index=False)

# Display results
print(results_df)


2024-11-17 12:02:59.672034: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-17 12:02:59.702014: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-17 12:03:01.178589: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-17 12:03:01.210448: I external/local_

Processing model: googlenet.h5 | Expected input shape: (299, 299)
Found 7928 images belonging to 2 classes.


2024-11-17 12:03:11.802354: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 90500


248/248 [==============================] - 29s 76ms/step
y_true shape: (7928,), y_pred shape: (7928, 1)
y_true sample: [0 0 0 0 0], y_pred sample: [[0]
 [0]
 [0]
 [0]
 [0]]
Processing model: nasnet_binary_classification_with_regularization.h5 | Expected input shape: (224, 224)
Found 7928 images belonging to 2 classes.
248/248 [==============================] - 21s 43ms/step
y_true shape: (7928,), y_pred shape: (7928, 1)
y_true sample: [0 0 0 0 0], y_pred sample: [[0]
 [0]
 [0]
 [0]
 [1]]
Processing model: resnet.h5 | Expected input shape: (320, 320)
Found 7928 images belonging to 2 classes.
248/248 [==============================] - 33s 121ms/step
y_true shape: (7928,), y_pred shape: (7928, 1)
y_true sample: [0 0 0 0 0], y_pred sample: [[0]
 [1]
 [1]
 [0]
 [1]]
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060, compute ca

2024-11-17 12:04:33.608277: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


Processing model: vgg16.h5 | Expected input shape: (224, 224)
Found 7928 images belonging to 2 classes.


2024-11-17 12:04:34.368601: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


248/248 [==============================] - 18s 62ms/step
y_true shape: (7928,), y_pred shape: (7928, 1)
y_true sample: [0 0 0 0 0], y_pred sample: [[0]
 [0]
 [0]
 [0]
 [0]]
Processing model: alexnet.h5 | Expected input shape: (227, 227)
Found 7928 images belonging to 2 classes.
248/248 [==============================] - 5s 20ms/step
y_true shape: (7928,), y_pred shape: (7928, 1)
y_true sample: [0 0 0 0 0], y_pred sample: [[0]
 [0]
 [1]
 [0]
 [0]]
Processing model: mobilenet.h5 | Expected input shape: (320, 320)
Found 7928 images belonging to 2 classes.
248/248 [==============================] - 13s 40ms/step
y_true shape: (7928,), y_pred shape: (7928, 1)
y_true sample: [0 0 0 0 0], y_pred sample: [[0]
 [0]
 [0]
 [0]
 [1]]
Processing model: lenet_binary_classification.h5 | Expected input shape: (32, 32)
Found 7928 images belonging to 2 classes.
248/248 [==============================] - 2s 9ms/step
y_true shape: (7928,), y_pred shape: (7928, 1)
y_true sample: [0 0 0 0 0], y_pred sample:

In [2]:
# For CNN_model

import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# Paths
models_folder = "/workspace/nudenet/CNN"
test_data_folder = "/workspace/nudenet/NudeNet_Classifier_train_data_x320/preprocessed/nude_sexy_safe_v1_x320/testing"

# Initialize results storage
results = []

# Loop through models
for model_name in os.listdir(models_folder):
    if model_name.endswith(".h5"):
        model_path = os.path.join(models_folder, model_name)
        
        # Load the model
        model = tf.keras.models.load_model(model_path)
        
        # Get model's expected input shape
        input_shape = model.input_shape[1:3]  # (height, width)
        print(f"Processing model: {model_name} | Expected input shape: {input_shape}")
        
        # Create test data generator with the model's input shape
        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
        test_data = test_datagen.flow_from_directory(
            test_data_folder,
            target_size=input_shape,
            batch_size=32,
            class_mode="binary",
            shuffle=False
        )
        
        # Predict
        y_true = test_data.classes
        y_pred_probs = model.predict(test_data)
        
        # Get the class with the maximum probability for each prediction
        y_pred = np.argmax(y_pred_probs, axis=1)  # This will give the class index (0 or 1)
        
        # Compute metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        roc_auc = roc_auc_score(y_true, y_pred_probs[:, 1])  # Using the second column for ROC-AUC
        
        # Append results
        results.append({
            "Model": model_name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1,
            "ROC-AUC": roc_auc
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save to CSV
results_csv_path = "model_evaluation_results.csv"
results_df.to_csv(results_csv_path, index=False)

# Display results
print(results_df)


Processing model: cnn.h5 | Expected input shape: (224, 224)
Found 7928 images belonging to 2 classes.
248/248 [==============================] - 5s 17ms/step
    Model  Accuracy  Precision    Recall  F1-Score   ROC-AUC
0  cnn.h5  0.820005   0.800734  0.862222  0.830341  0.895553
